# TAR: Taller de Aprendizaje por Refuerzo 2025
## Laboratorio 5: Gymnasium + stable_baselines3

### Stable Baselines3 (SB3) es una libreria algoritmos de reinforcement learning implementada en PyTorch. 

*Ejecutar esta celda solo la primera vez (si estan usando un entorno local - es la misma del lab 4) para descargar e instalar los paquetes necesarios. Si ejecutan el notebook en colab tendran que ejecutarla cada vez que reinicien el kernel*

In [ ]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

#### IMPORTS

In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Ejercicio 1. Lunar Landing

En este ejemplo entrenaremos un agente que aprenda a aterrizar correctamente una nave en la luna.
Doc del ambiente: https://gymnasium.farama.org/environments/box2d/lunar_lander/


#### 1.1 Estudiar el ambiente

In [ ]:
#TODO Definir el ambiente `LunarLander-v2` e imprimir el tamaño del espacio de observación y del espacio de acciones.

env = gym.make("LunarLander-v2", render_mode="human")
env.reset()

tamaño_obs = env.observation_space.shape  # Tamaño del espacio de observación
print("Tamaño del espacio de observación: ", tamaño_obs)

tamaño_acciones = env.action_space.n # ....
print("Tamaño del espacio de acción: ", tamaño_acciones)

El vector de **observación** corresponde a:

- x coordinate
- y coordinate
- vx
- vy
- angle
- w
- left leg on the ground or not
- right leg on the ground or not

Por otro lado, el espacio de **acciones** es:

- Acción 0: do nothing
- Acción 1: fire left orientation engine
- Acción 2: fire main engine
- Acción 3: fire right orientation engine

Función de **recompensa**:

  Después de cada paso, se otorga una recompensa. La recompensa total de un episodio es la suma de las recompensas de todos los pasos dentro de ese episodio.

  Para cada paso, la recompensa:

- es positiva si se acerca al lugar de aterrizaje y negativa si se aleja
- es positiva si la velocidad aumenta y negativa si disminuye
- es 10 para cada pata que este tocando el suelo
- es - 0.03 cada vez q usa un motor lateral
- es -0.3 cada vez q usa el motor principal
- 100 si aterriza correctamente
- -100 si se choca contra el piso

Un episodio termina si:
- si choca contra la luna
- si el lander se va de la pantalla
- si el lander no se mueve


##### Multi-procesamiento
A continuación se vectoriza el ambiente para poder paralelizar el entrenamiento, (por suerte la libreria nos da las heramientas para lograr esto sin mucho esfuerzo)

In [ ]:
from stable_baselines3.common.env_util import make_vec_env

# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

#### 1.2 Definir el modelo usando el modelo [Proximal Policy Optimization(PPO) de stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)

Leer la documentación y variar diferentes hiperparámetros como `batch_size`, `gamma`, entre otros.

¿Que tipo de metodo implementa?

##### RESPUESTA --> Policy based

In [ ]:
from stable_baselines3 import PPO

model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    device="cpu",
    verbose=1)

In [ ]:
# Entrenar por 1.000.000 timesteps

model.learn(total_timesteps=1000000)

model_name = "ppo-LunarLander-v2"

# Guardar el modelo
model.save(model_name)


#### 1.3 Evaluación del modelo:

¿Como podemos evaluar si el agente aprendió a aterrizar la nave?

##### RESPUESTA --> Una recompensa de 200 es señal de un buen aterrizaje

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(gym.make("LunarLander-v2", render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

#### 1.4 Correr el agente entrenado y guardar un video del aterrizaje

si esto da error de timeout pueden generar un .py y correrlo desde consola para guardar el video

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import RecordVideo

# Load the trained model
model = PPO.load("ppo-LunarLander-v2")

# Create the environment with the proper render_mode for video recording
env = DummyVecEnv([lambda: RecordVideo(gym.make("LunarLander-v2", render_mode="rgb_array"), "./videos/PPO", episode_trigger=lambda x: True)])

# Reset the environment to get the initial observation
obs = env.reset()
done = False

# Play one episode to capture the video
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
# Close the environment to save the video file
env.close()

print("Video saved in the './videos' directory.")


## Ejercicio 2. Cart Pole

En este ejemplo entrenaremos un agente que aprenda a equilibrar un poste sobre un carrito que se mueve sobre una linea.
Doc del ambiente: https://gymnasium.farama.org/environments/classic_control/cart_pole/


#### 2.1 Estudiar el ambiente:

¿Como son el espacio de acciones y el espacio de estados?

##### RESPUESTA -->
Acciones discretas, empujar a la derecha o izquierda cambia la velocidad del carro, pero depende del angulo de la barra el efecto de la accion.

Espacio de estados continuos (acotados) 4 dimensiones:
- posicion del carro
- velocidad del carro
- angulo de la barra
- velocidad angular de la barra


In [5]:
#TODO Definir el ambiente e imprimir el tamaño del espacio de observación y del espacio de acciones.
env= gym.make("CartPole-v1", render_mode="rgb_array")

env.reset()

tamaño_obs = env.observation_space.shape  # Tamaño del espacio de observación
print("Tamaño del espacio de observación: ", tamaño_obs)

tamaño_acciones = env.action_space.n # ....
print("Tamaño del espacio de acción: ", tamaño_acciones)

Tamaño del espacio de observación:  (4,)
Tamaño del espacio de acción:  2


#### 2.2 Definir el modelo usando el modelo - A2C de stable_baselines3

¿Que tipo de metodo implementa?

##### RESPUESTA --> Policy base, pero puede decirse que es combinado, porque aprende tmb una funcion de valor

In [8]:
from stable_baselines3 import A2C

model = A2C(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 5,
    gamma = 0.99,
    gae_lambda = 0.95,
    ent_coef = 0.01,
    device="cpu",
    verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


#### 2.3 Vectorizar el ambiente y entrenar el modelo

In [11]:
#TODO vectorizar el ambiente y entrenar el agente A2C
from stable_baselines3.common.env_util import make_vec_env

# Create the environment
env = make_vec_env('CartPole-v1', n_envs=16)

model = A2C(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 5,
    gamma = 0.99,
    gae_lambda = 0.95,
    ent_coef = 0.01,
    device="cpu",
    verbose=1)


model.learn(total_timesteps=1000000)

model_name = "A2C-CartPole-v1"

# Guardar el modelo
model.save(model_name)

Using cpu device
--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 30.8       |
|    ep_rew_mean        | 30.8       |
| time/                 |            |
|    fps                | 1162       |
|    iterations         | 100        |
|    time_elapsed       | 6          |
|    total_timesteps    | 8000       |
| train/                |            |
|    entropy_loss       | -0.584     |
|    explained_variance | 0.38004923 |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | 0.452      |
|    value_loss         | 14.7       |
--------------------------------------
--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 76.9       |
|    ep_rew_mean        | 76.9       |
| time/                 |            |
|    fps                | 1927       |
|    iterations         | 200        |
|    time_elapsed       | 8          |
|    tot

##### ¿Comó son las recompensas?
+1 por cada iteracion que la barra permanece sin caerse
##### ¿Comó se si mi modelo aprendió?
Si la recompensa es grande, pues el modelo recompensa el tiempo que permanece la barra sin caerse (sera el largo maximo de episodio

### RESPUESTA: 
en este caso una recompensa de 500 es un episodio exitoso

#### 2.4 Evaluar el agente


In [13]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import RecordVideo

# Load the trained model
model = A2C.load("A2C-CartPole-v1")

# Create the environment with the proper render_mode for video recording
env = DummyVecEnv([lambda: RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"), "./videos/A2C", episode_trigger=lambda x: True)])

# Reset the environment to get the initial observation
obs = env.reset()
done = False

# Play one episode to capture the video
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
# Close the environment to save the video file
env.close()

print("Video saved in the './videos' directory.")

MoviePy - Building video /home/mdelcastillo/proyectos/RL-curso/videos/A2C-/rl-video-episode-0.mp4.
MoviePy - Writing video /home/mdelcastillo/proyectos/RL-curso/videos/A2C-/rl-video-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /home/mdelcastillo/proyectos/RL-curso/videos/A2C-/rl-video-episode-0.mp4
MoviePy - Building video /home/mdelcastillo/proyectos/RL-curso/videos/A2C-/rl-video-episode-1.mp4.
MoviePy - Writing video /home/mdelcastillo/proyectos/RL-curso/videos/A2C-/rl-video-episode-1.mp4



MoviePy - Done !
MoviePy - video ready /home/mdelcastillo/proyectos/RL-curso/videos/A2C-/rl-video-episode-1.mp4
Video saved in the './videos' directory.
